In [1]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

print("Training data")
train_df.head()

Training data


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
# --- 1. Обработка 'Age' (Возраст) ---
# 'Безопасный' способ: мы присваиваем результат обратно колонке.
median_age = train_df['Age'].median() # Сначала найдем медиану
train_df['Age'] = train_df['Age'].fillna(median_age)
test_df['Age'] = test_df['Age'].fillna(median_age)

# --- 2. Обработка 'Embarked' (Порт посадки) ---
mode_embarked = train_df['Embarked'].mode()[0] # Найдем самое частое значение
train_df['Embarked'] = train_df['Embarked'].fillna(mode_embarked)
test_df['Embarked'] = test_df['Embarked'].fillna(mode_embarked)

# --- 3. Обработка 'Fare' (Цена билета) ---
median_fare = test_df['Fare'].median() # Найдем медиану цены
test_df['Fare'] = test_df['Fare'].fillna(median_fare)

# --- 4. Превращаем текст в числа ('Sex') ---
# Этот код был в порядке, но для единообразия напишем его так же
train_df['Sex'] = train_df['Sex'].map({'male': 0, 'female': 1})
test_df['Sex'] = test_df['Sex'].map({'male': 0, 'female': 1})

# --- 5. Превращаем текст в числа ('Embarked') ---
# Этот код тоже был в порядке
train_df = pd.get_dummies(train_df, columns=['Embarked'], drop_first=True)
test_df = pd.get_dummies(test_df, columns=['Embarked'], drop_first=True)

# --- 6. НОВАЯ ФИЧА: 'Title' (Титул) ---
# Извлекаем титулы ('Mr.', 'Miss.', 'Mrs.') из колонки 'Name'
train_df['Title'] = train_df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
test_df['Title'] = test_df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# --- 7. НОВАЯ ФИЧА: 'FamilySize' (Размер семьи) ---
# Размер семьи = Он сам (1) + Братья/Супруги (SibSp) + Родители/Дети (Parch)
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1
test_df['FamilySize'] = test_df['SibSp'] + test_df['Parch'] + 1

# --- 8. НОВАЯ ФИЧА: 'IsAlone' (Одиночка) ---
# Создаем бинарный признак: 1 = Одиночка, 0 = Есть семья
train_df['IsAlone'] = 0
train_df.loc[train_df['FamilySize'] == 1, 'IsAlone'] = 1

test_df['IsAlone'] = 0
test_df.loc[test_df['FamilySize'] == 1, 'IsAlone'] = 1

print("Созданы фичи 'FamilySize' и 'IsAlone'.")

# Группируем редкие титулы в одну категорию 'Rare'
common_titles = ['Mr', 'Miss', 'Mrs', 'Master']
# .apply() - это как цикл. Если титул НЕ в списке common_titles, меняем его на 'Rare'
train_df['Title'] = train_df['Title'].apply(lambda x: x if x in common_titles else 'Rare')
test_df['Title'] = test_df['Title'].apply(lambda x: x if x in common_titles else 'Rare')

# Теперь превращаем текстовые титулы ('Mr', 'Miss'...) в числа (1, 2, 3...)
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
train_df['Title'] = train_df['Title'].map(title_mapping)
test_df['Title'] = test_df['Title'].map(title_mapping)

# На всякий случай заполним пропуски (если титул не нашелся)
train_df['Title'] = train_df['Title'].fillna(0)
test_df['Title'] = test_df['Title'].fillna(0)

print("Создана новая фича 'Title'.")

train_df.head()

Созданы фичи 'FamilySize' и 'IsAlone'.
Создана новая фича 'Title'.


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_Q,Embarked_S,Title,FamilySize,IsAlone
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,False,True,1,2,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,False,False,3,2,0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,False,True,2,1,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,False,True,3,2,0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,False,True,1,1,1


In [3]:
# --- 2. ВЫБОР ПРИЗНАКОВ И ОБУЧЕНИЕ НОВОЙ МОДЕЛИ ---

# --- ИМПОРТИРУЕМ НОВУЮ МОДЕЛЬ ---
from sklearn.ensemble import RandomForestClassifier

# --- ДОБАВЛЯЕМ НОВЫЕ ФИЧИ В СПИСОК ---
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_Q', 'Embarked_S', 'Title', 'FamilySize', 'IsAlone']

print("--- МОДЕЛЬ БУДЕТ УЧИТЬСЯ НА ЭТИХ ПРИЗНАКАХ: ---")
print(features)

target = 'Survived'

X_train = train_df[features]
y_train = train_df[target]

# В X_test могут быть колонки, которых нет в X_train (например, если в test не было Embarked_Q)
# Этот код гарантирует, что в X_test те же колонки, что и в X_train
X_test = test_df[features]

# --- СОЗДАЕМ И ОБУЧАЕМ НОВУЮ МОДЕЛЬ RandomForest ---
# n_estimators=100 - "100 деревьев в лесу"
# max_depth=5 - ограничиваем глубину, чтобы не переобучиться
# random_state=1 - чтобы результат был воспроизводимым
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X_train, y_train)

print("Модель RandomForest ОБУЧЕНА!")

--- МОДЕЛЬ БУДЕТ УЧИТЬСЯ НА ЭТИХ ПРИЗНАКАХ: ---
['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_Q', 'Embarked_S', 'Title', 'FamilySize', 'IsAlone']
Модель RandomForest ОБУЧЕНА!


In [4]:
# --- 1. Делаем предсказания ---
predictions = model.predict(X_test)

# --- 2. Готовим файл для отправки ---
# Kaggle требует файл в формате: две колонки (PassengerId и Survived)
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': predictions
})

# --- 3. Сохраняем файл ---
# Файл сохранится в ту же папку, где лежит твой ноутбук (т.е. в папку 'notebook')
submission.to_csv('submission_V4_RandomForest.csv', index=False)

print("Создан V4 файл с RandomForest!")
submission.head()

Создан V4 файл с RandomForest!


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
